In [12]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame as df
import seaborn as sns


from keras.layers import LSTM, Dense
from keras.models import Sequential
import keras.backend as K
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold, cross_val_score

from sklearn.model_selection import train_test_split

In [3]:
data_1 = glob.glob('C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우/*.csv')
data_2 = glob.glob('C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌/*.csv')
data_3 = glob.glob('C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계방향/*.csv')
data_4 = glob.glob('C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향/*.csv')
len(data_1), len(data_2), len(data_3), len(data_4)

(100, 100, 100, 100)

In [9]:
total_data = data_1 + data_2 + data_3 + data_4

dataset = np.zeros((len(total_data), 150, 2))
dataset

array([[[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       ...,

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]]])

In [10]:
for i, data in enumerate(total_data):
    print(data)
    dataset[i] = pd.read_csv(data, names=['a-b','c-d','label']).sample(n=150).sort_index().iloc[:,:2]

C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\1.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\10.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\100.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\11.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\12.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\13.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\14.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\15.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\16.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\17.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\18.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\19.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\2.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\20.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\21.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\22.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\23.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\24.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/좌-우\2

C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\78.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\79.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\8.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\80.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\81.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\82.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\83.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\84.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\85.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\86.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\87.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\88.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\89.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\9.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\90.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\91.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\92.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\93.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/우-좌\94

C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\48.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\49.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\5.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\50.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\53.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\54.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\55.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\56.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\57.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\58.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\59.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\6.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\60.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\61.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\62.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\63.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\64.csv
C:/Users/hml76/OneDrive/문서/카카오톡 받은 파일/논문용/시계반대방향\6

In [11]:
label = np.array([[1, 0, 0 ,0 ]] * len(data_1) + [[0, 1, 0, 0]] * len(data_2) + [[0, 0, 1, 0]] * len(data_3) + [[0, 0, 0, 1]] * len(data_4))
dataset.shape, label.shape

((400, 150, 2), (400, 4))

In [27]:
X_train, X_test, y_train, y_test = train_test_split(dataset, label, random_state=0, test_size=0.2)
X_test.shape

(80, 150, 2)

In [16]:
callback = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.000001), 
    #ModelCheckpoint(filepath='./weights.h5', save_best_only=True),
    EarlyStopping(monitor='val_acc', restore_best_weights=True, patience=5)
]

In [17]:
K.clear_session()

In [21]:
model_once = Sequential()

model_once.add(LSTM(20, input_shape=(150,2)))
model_once.add(Dense(4, activation='softmax'))
model_once.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.005), metrics=['acc'])

model_once.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 20)                1840      
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 84        
Total params: 1,924
Trainable params: 1,924
Non-trainable params: 0
_________________________________________________________________


In [22]:
model_once.fit(X_train, y_train, epochs=30)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/30
320/320 [==============================] - 3s 8ms/step - loss: 1.3562 - acc: 0.4875
Epoch 2/30
320/320 [==============================] - 1s 3ms/step - loss: 1.2258 - acc: 0.5000
Epoch 3/30
320/320 [==============================] - 1s 3ms/step - loss: 1.0462 - acc: 0.5344
Epoch 4/30
320/320 [==============================] - 1s 4ms/step - loss: 0.9003 - acc: 0.5750
Epoch 5/30
320/320 [==============================] - 1s 3ms/step - loss: 0.8131 - acc: 0.7094
Epoch 6/30
320/320 [==============================] - 1s 3ms/step - loss: 0.7294 - acc: 0.7781
Epoch 7/30
320/320 [==============================] - 1s 4ms/step - loss: 0.7167 - acc: 0.7937
Epoch 8/30
320/320 [==============================] - 1s 3ms/step - loss: 0.5695 - acc: 0.8344
Epoch 9/30
320/320 [==============================] - 1s 3ms/step - loss: 1.0748 - acc: 0.6063
Epoch 10/30
320/320 [==============================

In [23]:
model_once.evaluate(X_test, y_test)

80/80 [==============================] - 0s 3ms/step


[0.28929988145828245, 0.925]

In [25]:
y_test_to_int = np.arange(len(y_test))
x_test_to_int = np.arange(len(X_test)) 

for j in range(len(X_test)):
    x_test_to_int[j] = int(model_once.predict_classes(X_test[j:j+1]))
    for k in range(4):
        if(y_test[j,k] == 1):
            y_test_to_int[j] = k 

y_test_to_int[0:20], x_test_to_int[0:20]           

(array([1, 3, 3, 1, 2, 0, 1, 2, 1, 2, 0, 1, 1, 3, 0, 0, 2, 1, 2, 1]),
 array([1, 3, 0, 1, 2, 0, 1, 2, 1, 2, 0, 1, 1, 2, 0, 0, 2, 1, 2, 1]))

In [26]:
count=0
print("Real value : Prediction\n---------------------------")
for i in range(len(X_test)):
    if(y_test_to_int[i] != x_test_to_int[i]):
        print("   ",y_test_to_int[i], "\t   : \t", x_test_to_int[i])
        count = count + 1
probab = count/len(X_test)
print("\n\nNumber of miscalculation : ",count,"/",len(x_test_to_int),"=",round(probab,3))
print("\nAccuracy : ",((1-round(probab,4))*100),"%")

Real value : Prediction
---------------------------
    3 	   : 	 0
    3 	   : 	 2
    2 	   : 	 1
    0 	   : 	 3
    0 	   : 	 3
    0 	   : 	 3


Number of miscalculation :  6 / 80 = 0.075

Accuracy :  92.5 %


In [29]:
test_data_set_CW1 = pd.read_csv("C:/Users/hml76/OneDrive/바탕 화면/122.csv",names=['a-b','c-d'])
test_data_set_CW1 = np.zeros((2, 150, 2))
test_data_set_CW1
for i, data in enumerate(test_data_set_CW1):
    test_data_set_CW1[i] = pd.read_csv("C:/Users/hml76/OneDrive/바탕 화면/122.csv", names=['a-b','c-d']).sample(n=150).sort_index().iloc[:,:2]
test_data_set_CW1    
model_once.predict_classes(test_data_set_CW1)

array([2, 2], dtype=int64)